In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Model Loading

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 5000 # Should be sufficient for three full articles + instruction + roundup
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "/content/drive/Shareddrives/FYP 2024-2025/Phase-2/SummaryGen/models/phi-3.5-lora-r-16", # YOUR MODEL YOU USED FOR TRAINING
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# ) # load saved model for inference later if needed

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16 ,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.



## Data Preparation


In [ ]:
import json
import pandas as pd
from collections import defaultdict

input_file = "/content/drive/Shareddrives/FYP 2024-2025/Phase-2/AllSides/allsides_clean_sentences_quotes.jsonl"

# Load records
records = []
with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        records.append(json.loads(line.strip()))

# Convert to DataFrame
df = pd.DataFrame(records)
print("Size of df: ", df.shape)
df.head()

Size of df:  (430, 7)


,id,issue,topic,link,date,roundup,news
0,4009,DOJ Rejects McCabe's Appeal to Avoid Charges,FBI,/story/doj-rejects-mccabes-appeal-avoid-charges,2019-09-12T14:30:00-07:00,[The Department of Justice rejected former FBI...,"{'left': {'stance': 'left', 'stance_specific':..."
1,4015,Thousands Walk Off GM Jobs as UAW Strike Starts,Labor,/story/thousands-walk-job-uaw-strike-starts,2019-09-16T10:45:00-07:00,[After negotiations for a new contract fell th...,"{'left': {'stance': 'left', 'stance_specific':..."
2,4017,Pressley To Push For Kavanaugh Impeachment Inq...,US House,/story/representative-ayanna-pressley-push-imp...,2019-09-17T07:45:00-07:00,[Representative Ayanna Pressley (D-MA) is plan...,"{'left': {'stance': 'left', 'stance_specific':..."
3,4021,Federal Reserve Cuts Interest Rates for Second...,Economy and Jobs,/story/federal-reserve-cuts-interest-rates-sec...,2019-09-18T16:00:00-07:00,[The Federal Reserve reduced its benchmark len...,"{'left': {'stance': 'left', 'stance_specific':..."
4,4023,Whistleblower Alleges Trump Had Inappropriate ...,White House,/story/whistleblower-alleges-trump-had-inappro...,2019-09-19T11:00:00-07:00,[A government watchdog alleges that President ...,"{'left': {'stance': 'left', 'stance_specific':..."


In [ ]:
def group_paragraphs(sentences, paragraph_positions):
    """
    Group each sentence with its paragraph index from paragraph_positions.
    - Join sentences within the same paragraph with a space.
    - Separate paragraphs with a newline.
    """
    paragraphs_map = defaultdict(list)
    for sentence, ppos in zip(sentences, paragraph_positions):
        paragraphs_map[ppos].append(sentence.strip())

    # Sort by paragraph index and join sentences with a space
    # Then join paragraphs with newlines
    sorted_paragraphs = []
    for ppos in sorted(paragraphs_map.keys()):
        paragraph_text = " ".join(paragraphs_map[ppos])
        sorted_paragraphs.append(paragraph_text)
    return "\n".join(sorted_paragraphs)

def build_conversation(row):
    """
    Constructs a conversation with:
      1) A single-line 'system' message
      2) A 'human' message containing the event + all articles
      3) An 'assistant' message containing the target neutral roundup
    """
    # Single-line system prompt
    system_message = "You are given an event headline and news articles from the left, right, and center perspectives. Your task is to summarize the event in three to four sentences, and mention how different the different perspectives covered the event, to do this, you can highlight what each perspective emphasized on. This summary must be factual and neutral without any bias."

    # Gather event
    event_title = row.get("issue", "No Event Title")

    # Build left, center, right texts using paragraph_pos
    # (If 'left'/'center'/'right' exist in row["news"] and each has 'newsContent' + 'paragraph_pos')
    left_data = row["news"]["left"]
    left_text = group_paragraphs(left_data["newsContent"], left_data["paragraph_pos"]) if left_data else ""

    center_data = row["news"]["center"]
    center_text = group_paragraphs(center_data["newsContent"], center_data["paragraph_pos"]) if center_data else ""

    right_data = row["news"]["right"]
    right_text = group_paragraphs(right_data["newsContent"], right_data["paragraph_pos"]) if right_data else ""

    # Combine into a single 'human' message
    human_message = (
        f"### Event Headline\n{event_title}\n"
        f"### Left Article\n{left_text}\n"
        f"### Center Article\n{center_text}\n"
        f"### Right Article\n{right_text}"
    )

    # Target neutral roundup (assistant message)
    if isinstance(row["roundup"], list):
        assistant_message = "".join(row["roundup"])
    else:
        assistant_message = str(row["roundup"])

    return [
        {"from": "system", "value": system_message},
        {"from": "human", "value": human_message},
        {"from": "assistant", "value": assistant_message}
    ]

In [ ]:
# Create 'conversations' column
df['conversations'] = df.apply(build_conversation, axis=1)
print(df[['conversations']].head())

                                       conversations
0  [{'from': 'system', 'value': 'You are given an...
1  [{'from': 'system', 'value': 'You are given an...
2  [{'from': 'system', 'value': 'You are given an...
3  [{'from': 'system', 'value': 'You are given an...
4  [{'from': 'system', 'value': 'You are given an...


In [ ]:
# system value
print(df['conversations'][0][0]['value'])

You are given an event headline and news articles from the left, right, and center perspectives. Your task is to summarize the event in three to four sentences, and mention how different the different perspectives covered the event, to do this, you can highlight what each perspective emphasized on. This summary must be factual and neutral without any bias.


In [ ]:
# human value
print(df['conversations'][0][1]['value'])

### Event Headline
DOJ Rejects McCabe's Appeal to Avoid Charges
### Left Article
WASHINGTON — The Justice Department rejected a last-ditch appeal from defense lawyers for the former F.B.I. deputy director Andrew G. McCabe, people familiar with the matter said on Thursday, as prosecutors have appeared close to charging him in a criminal case.
Lawyers for Mr. McCabe, who is being investigated over whether he lied to internal investigators about dealings with the news media, had argued that prosecutors lacked evidence to charge him. The lawyers detailed their position to top officials including Jeffrey A. Rosen, the deputy attorney general, who ultimately rejected their view. His deputy, Edward O’Callaghan, who was also present during the discussion, notified Mr. McCabe’s lawyers of the decision, one of the people said.
The case is politically fraught for the Justice Department because of President Trump’s repeated attacks on Mr. McCabe, who was the acting director of the F.B.I. when inve

In [ ]:
# assistant value
print(df['conversations'][0][2]['value'])

The Department of Justice rejected former FBI deputy director Andrew McCabe's attempts to avoid prosecution.McCabe is accused of lying to investigators about media leaks involving Hillary Clinton and the 2016 Presidential election.Some left-leaning reporting painted McCabe's indictment as unlikely; others, especially right-leaning, reporting labeled an indictment as "imminent."


In [ ]:
from sklearn.model_selection import train_test_split

train_conversations, temp_conversations = train_test_split(
    df['conversations'],
    test_size=0.1,
    random_state=42
)

val_conversations, test_conversations = train_test_split(
    temp_conversations,
    test_size=0.5,
    random_state=42
)

print(f"Train set size: {len(train_conversations)}")
print(f"Validation set size: {len(val_conversations)}")
print(f"Test set size: {len(test_conversations)}")

train_df = pd.DataFrame(train_conversations, columns=['conversations'])
val_df = pd.DataFrame(val_conversations, columns=['conversations'])
test_df = pd.DataFrame(test_conversations, columns=['conversations'])

Train set size: 387
Validation set size: 21
Test set size: 22


In [ ]:
from datasets import Dataset
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
test_ds = Dataset.from_pandas(test_df)

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3",
    mapping = {"role" : "from", "content" : "value", "user" : "user", "assistant" : "assistant", "system" : "system"}, # ShareGPT style
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

train_dataset = train_ds.map(formatting_prompts_func, batched = True,)
val_dataset = val_ds.map(formatting_prompts_func, batched = True,)
test_dataset = test_ds.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/387 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

In [ ]:
train_dataset[1]["conversations"]

[{'from': 'system',
  'value': 'You are given an event headline and news articles from the left, right, and center perspectives. Your task is to summarize the event in three to four sentences, and mention how different the different perspectives covered the event, to do this, you can highlight what each perspective emphasized on. This summary must be factual and neutral without any bias.'},
 {'from': 'human',
  'value': '### Event Headline\nTrump Defense Team Prepares For Final Day of Opening Arguments\n### Left Article\nOn Wednesday afternoon, the next phase of the trial begins — a chance for senators to ask questions of the prosecution and defense. They’ll spend up to eight hours a day asking questions, alternating between Republican and Democratic senators. Here’s the catch: The Senate rules passed in the 1980s outlining how to do a trial say senators have to ask these questions in writing (to avoid grandstanding). The chief justice will read them out loud.\n### Center Article\nSena

In [ ]:
print(train_dataset[1]["text"])

<|system|>
You are given an event headline and news articles from the left, right, and center perspectives. Your task is to summarize the event in three to four sentences, and mention how different the different perspectives covered the event, to do this, you can highlight what each perspective emphasized on. This summary must be factual and neutral without any bias.<|end|>
<|human|>
### Event Headline
Trump Defense Team Prepares For Final Day of Opening Arguments
### Left Article
On Wednesday afternoon, the next phase of the trial begins — a chance for senators to ask questions of the prosecution and defense. They’ll spend up to eight hours a day asking questions, alternating between Republican and Democratic senators. Here’s the catch: The Senate rules passed in the 1980s outlining how to do a trial say senators have to ask these questions in writing (to avoid grandstanding). The chief justice will read them out loud.
### Center Article
Senate Majority Leader Mitch McConnell told a g

## Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 5,
        evaluation_strategy="steps",
        eval_steps=10,
        max_steps = 120,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
        load_best_model_at_end = True
    ),
     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/387 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/21 [00:00<?, ? examples/s]

In [ ]:
# r-16 # ran for 120 steps with continually decreasing val loss
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 387 | Num Epochs = 3 | Total steps = 120
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416/4,000,000,000 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
10,2.573200,2.492267
20,2.374000,2.257163
30,2.240200,2.051112
40,1.958200,1.926180
50,1.919600,1.855238
60,1.933200,1.817837
70,1.897800,1.795848
80,1.730000,1.782734


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Step,Training Loss,Validation Loss
10,2.573200,2.492267
20,2.374000,2.257163
30,2.240200,2.051112
40,1.958200,1.926180
50,1.919600,1.855238
60,1.933200,1.817837
70,1.897800,1.795848
80,1.730000,1.782734
90,1.639000,1.774699
100,1.778900,1.768881


### Inference

Example outputs generating the need for model enhancement with RLHF

Content Preservation: Model oversimplifies and presents wrong content due to hallucination

In [ ]:
from unsloth.chat_templates import get_chat_template

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

message = [
    test_dataset[3]['conversations'][0],
    test_dataset[3]['conversations'][1],
]
inputs = tokenizer.apply_chat_template(
    message,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True)
text_output = tokenizer.batch_decode(outputs)[0]
text_output = text_output.split("<|assistant|>")[1].strip()
text_output = text_output.split("<|end|>")[0].strip()
print(text_output)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


The U.S. economy added 148,00s in December, according to the Labor Department’s report. The unemployment rate remained steady at 4.4 percent.


In [ ]:
# Expert written roundup
test_dataset[3]['conversations'][2]['value']

'The U.S. added 148,000 jobs in Dec., falling short of the 190,000 jobs expected.Unemployment remained at 4.1%, which is a 17 year low.'

In [ ]:
# News articles
print(test_dataset[3]['conversations'][1]['value'])

### Event Headline
December 2017 Jobs Report
### Left Article
The U.S. economy added 148,000 jobs in December after a year of steady hiring, missing expectations for a larger last-minute surge, the government reported Friday. The unemployment rate stayed at 4.1 percent last month, the lowest point since 2000. Wages continued their slow climb, rising by 9 cents to $26.63. That’s a 2.5 percent increase since December 2016 (and still below pre-recession levels).
President Trump's first year in the White House brought healthy growth and 2.1 million new jobs, a slight drop from 2.2 million positions created during Obama's last year in office, government estimates showed Friday.
The average number of jobs employers added each month in 2017 was 173,000, compared to the previous year’s 187,000.
One dark spot was retail, which lost 20,000 jobs in December, according to the Bureau of Labor Statistics. About 67,000 positions vanished from stores in 2017, a sharp reversal from the 203,000 jobs tha

In [ ]:
from unsloth.chat_templates import get_chat_template

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

message = [
    test_dataset[2]['conversations'][0],
    test_dataset[2]['conversations'][1],
]
inputs = tokenizer.apply_chat_template(
    message,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True)
text_output = tokenizer.batch_decode(outputs)[0]
text_output = text_output.split("<|assistant|>")[1].strip()
text_output = text_output.split("<|end|>")[0].strip()

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
text_output

"President Trump and Japanese Prime Minister Shinzo Abe met in Tokyo on Sunday, and discussed North Korea's nuclear and missile programs, as well as trade and security issues."

In [ ]:
# Expert-written roundup
test_dataset[2]['conversations'][2]['value']

'President Trump kicked off his 12 day trip to Asia in Japan meeting with Japanese Prime Minister Shinzo Abe to discuss trade and North Korea.President Trump has also said he would probably meet with Russian President Vladimir Putin while on the trip to discuss North Korea.'

In [ ]:
# News articles covering the event
print(test_dataset[2]['conversations'][1]['value'])

### Event Headline
Trump Kicks Off Asia Trip
### Left Article
TOKYO — President Trump kicked off an extensive swing through Asia with a campaign-style rally on Sunday with American troops in Japan, as he looked toward a lengthy trip likely to be dominated by talks on confronting the nuclear threat from North Korea.
In a speech after Air Force One landed at Yokota Air Base in Tokyo on a crisp, sunny morning, Mr. Trump never mentioned Pyongyang by name. Sounding a militaristic tone, he sought to project toughness in the face of global challenges, saying the United States armed forces stood ready to defend itself and its allies and “fight to overpower” its adversaries.
“No one — no dictator, no regime and no nation — should underestimate, ever, American resolve,” Mr. Trump said, having shed his suit jacket for a leather bomber jacket as he addressed hundreds of fatigues-clad women and men. “You are the greatest threat to tyrants and dictators who seek to prey on the innocent.”
Breaking wi

Polarity bias in generated roundups: Model uses one article heavily to generate the summary

In [ ]:
from unsloth.chat_templates import get_chat_template

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

message = [
    test_dataset[19]['conversations'][0],
    test_dataset[19]['conversations'][1],
]
inputs = tokenizer.apply_chat_template(
    message,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True)
text_output = tokenizer.batch_decode(outputs)[0]
text_output = text_output.split("<|assistant|>")[1].strip()
text_output = text_output.split("<|end|>")[0].strip()

In [ ]:
# model output
text_output

'President Donald Trump slammed social media and big tech companies for their censorship of conservative voices, arguing that mainstream media sites like CNN and MSNBC are allowed to post fake news without punishment, while conservatives are being silenced.'

In [ ]:
# expert-written neutral summary
test_dataset[19]['conversations'][2]['value']

'On Saturday, President Trump tweeted criticism of social media companies, saying they are censoring and discriminating against Republican and conservative voices by shutting down their accounts or otherwise removing their content.'

In [ ]:
# news articles covering the event
print(test_dataset[19]['conversations'][1]['value'])

### Event Headline
President Trump Criticizes Social Media Companies for "Censorship" and "Discrimination" Against the Right
### Left Article
“Let everybody participate, good and bad, and we will all just have to figure it out,” the president tweeted Saturday morning.
President Donald Trump used Twitter to tear into social media companies Saturday morning, claiming they were “totally discriminating against Republican/Conservative voices.”
But he does have a solution. Sort of. And it involves both sides.
Trump tweeted that “too many voices are being destroyed, some good and some bad, and that cannot be allowed to happen.” So, he added: “Let everybody participate, good & bad, and we will all just have to figure it out!”
Check out his tweets here:
Social Media is totally discriminating against Republican/Conservative voices. Speaking loudly and clearly for the Trump Administration, we won’t let that happen. They are closing down the opinions of many people on the RIGHT, while at the same 

### Inference on Test

In [ ]:
from unsloth.chat_templates import get_chat_template
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def generate_roundups(test_dataset):
  eval = []
  for i in range(len(test_dataset)):
      message = [
          test_dataset[i]['conversations'][0],
          test_dataset[i]['conversations'][1],
      ]
      inputs = tokenizer.apply_chat_template(
          message,
          tokenize = True,
          add_generation_prompt = True, # Must add for generation
          return_tensors = "pt",
      ).to("cuda")
      outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True)
      text_output = tokenizer.batch_decode(outputs)[0]
      text_output = text_output.split("<|assistant|>")[1].strip()
      text_output = text_output.split("<|end|>")[0].strip()
      record = {'model_output': text_output, 'target_roundup': test_dataset[i]['conversations'][2]['value']}
      eval.append(record)
  return eval

eval = generate_roundups(test_dataset)

In [ ]:
eval_df = pd.DataFrame(eval)
eval_df.to_excel('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/model-evaluation/roundup_gen.xlsx', index=False)

In [ ]:
eval_df

,model_output,target_roundup
0,The Trump administration has revised the publi...,"Ken Cuccinelli, head of Citizenship and Immigr..."
1,The German authorities have released a man in ...,The German authorities say they cannot be sure...
2,President Trump and Japanese Prime Minister Sh...,President Trump kicked off his 12 day trip to ...
3,"The U.S. economy added 148,000 jobs in Decembe...","The U.S. added 148,000 jobs in Dec., falling s..."
4,The U.S. Treasury Secretary has announced that...,Treasury Secretary Steven Mnuchin said Tuesday...
5,President Trump is the most admired man in 2020.,President Donald Trump and former first lady M...
6,"The U.S. economy added 245,000 jobs in Novembe...",U.S. job growth slowed significantly in Novemb...
7,The top U.S. nuclear commander said he would d...,The top nuclear commander in the U.S. said he ...
8,Memorial Day is a day to honor the fallen sold...,"On May 29, 2017, people from all across the ai..."
9,Chief Justice John Roberts spoke out after a h...,"Chief Justice John Roberts, who is currently p..."


### Save model

In [ ]:
model.save_pretrained("/content/drive/Shareddrives/FYP 2024-2025/Phase-2/SummaryGen/models/phi-3.5-lora-r-16")
tokenizer.save_pretrained("/content/drive/Shareddrives/FYP 2024-2025/Phase-2/SummaryGen/models/phi-3.5-lora-r-16")

('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/SummaryGen/models/phi-3.5-lora-r-16/tokenizer_config.json',
 '/content/drive/Shareddrives/FYP 2024-2025/Phase-2/SummaryGen/models/phi-3.5-lora-r-16/special_tokens_map.json',
 '/content/drive/Shareddrives/FYP 2024-2025/Phase-2/SummaryGen/models/phi-3.5-lora-r-16/tokenizer.model',
 '/content/drive/Shareddrives/FYP 2024-2025/Phase-2/SummaryGen/models/phi-3.5-lora-r-16/added_tokens.json',
 '/content/drive/Shareddrives/FYP 2024-2025/Phase-2/SummaryGen/models/phi-3.5-lora-r-16/tokenizer.json')